<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/%C4%90%E1%BB%93_%C3%A1n_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tóm tắt bài toán

Input: một video bất kì

Output: phát hiện những cảnh đánh nhau (nếu có) trong video


#Thu thập dữ liệu

Dữ liệu nhóm tự thu thập bằng việc download các video có cảnh đánh nhau được camera quan sát ghi lại trên mạng xã hội như facebook, youtube…. Với tổng số khoảng gần 200 video.

Tiến hành chia video thành các đoạn clip nhỏ với 90 khung hình trên thời lượng 3s.

Sàng lọc, lựa ra những clip chỉ chứa cảnh đánh nhau và clip không chứa cảnh đánh nhau.

Tổng số clip sau khi sàng lọc từ việc cắt video là 817 clip trong đó:

+ 405 clip chỉ chứa cảnh đánh nhau.

+ 412 clip không chứa cảnh đánh nhau.

#Tiền xử lý Dữ liệu

## Import các thư viện cần thiết

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from skimage.feature import hog
from sklearn.cluster import KMeans
from os import listdir
from os.path import isfile, join

from sklearn.metrics import precision_recall_fscore_support

##Trích xuất Optical flow bằng thuật toán TVL1 có trong OpenCV
 

### Định nghĩa về Optical flow
Optical Flow là khái niệm chỉ sự chuyển động tương đối của các điểm trên bề mặt một đối tượng, vật thể nào đó gây ra, dưới góc quan sát của một điểm mốc (mắt, camera…).

Dễ hiểu là biểu diễn điểm khác biệt giữa 2 khung hình do vật thể di chuyển gây ra.

![Hình ảnh mô tả Optical flow của các xe](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/opticalflow_lk.jpg)

###Các bước thực hiện
B1: Định nghĩa đối tượng bằng hàm DualTVL1OpticalFlow_create()

B2: Tính flowx, flowy của 2 frame gần nhau bằng hàm calc. Với flowx là Optical Flow theo hướng x và tương tự với y

B3: Tính mag = sqrt(flowx + flowy)

B4: Normalize mag về ảnh hsv

B5: Chuyển ảnh về định dạng Grayscale

In [ ]:
optical_flow = cv2.optflow_DualTVL1OpticalFlow.create()
def OpticalFlow(prvs, next, hsv):
    
    flow = optical_flow.calc(prvs, next, None)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    nom = np.zeros((160,120))
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    return gray

##Trích xuất vùng chuyển động
Từ ảnh Optimal Flow vừa tìm được ta trích xuất ra vùng có sự chuyển động


###Phát hiện cạnh với thuật toán Canny
![Cận trên và dứoi](https://docs.opencv.org/trunk/hysteresis.jpg)

Thuật toán Canny tham khảo thêm [tại đây](https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html)

Hàm tìm maxVal và minVal tham khảo thêm [tại đây](https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/)

In [ ]:
def CvCanny(matrix):
  v = np.median(matrix)
  sigma = 0.33
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  canny = cv2.Canny(matrix, lower, upper)
  return canny

###Trích xuất vùng chuyển động

In [ ]:
def MRM(prvs, next, hsv):
    #Trích xuất dòng quang từ 2 frame gần nhau
    gray = OpticalFlow(prvs, next, hsv)

    #Phát hiện cạnh
    canny = CvCanny(gray)

    #Đóng các nét hở và lấp đầy
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

    #Lấp đầy những lỗ hổng trong vùng chuyển động
    contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv2.drawContours(closing,[cnt],0,255,-1)
    
    #Xóa bỏ những nét thừa
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    
    return opening

#Trích xuất đặc trưng


##Vùng quan tâm

Xem vùng chuyển động như là một vùng quan tâm (ROI), áp dụng ROI vào ảnh xám

In [ ]:
def apply_roi(img, roi):
  #thresh, roi = cv2.threshold(roi, thresh=128, maxval=1, type=cv2.THRESH_BINARY)
  new_img = img * roi
  return new_img

##Trích xuất đặc trưng

Đọc các clip dài 3s một vào.

Cứ 6 frame lấy 1 frame, trích xuất dòng quang giữa 2 frame kề nhau

Sử dụng HOG để trích xuất đặc trưng từ ảnh xám đã được gắn ROI

In [ ]:
def extract_features(path, label, descriptor_list, labels):
    #Lấy danh sách các clip trong path 
    videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    for videoPath in videoPaths:

        #Đọc video
        cap = cv2.VideoCapture(videoPath)
        videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

        ret, frame1 = cap.read()
        if not ret:
          break;
        #Thay đổi kích thước frame
        frame1 = cv2.resize(frame1, (160, 120))
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255
        s = 1
        while(cap.isOpened()):
            ret, frame2 = cap.read()

            if not ret:
                break;

            if s % 6 == 0:
                frame2 = cv2.resize(frame2, (160, 120))
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                
                #trích xuất vùng chuyển động giữa hai frame
                mrm = MRM(prvs, next, hsv)

                #Áp dụng ROI vào ảnh xám
                after_roi = apply_roi(next, mrm)

                #Trích xuất đặc trưng bằng HOG
                descriptor = hog(after_roi, orientations=12, pixels_per_cell=(4, 4), cells_per_block=(2,2), block_norm='L2-Hys', visualize=False)
                descriptor_list.append(descriptor)
                
                #Gắn nhãn
                labels.append(label)
                
                #Lưu frame hiện tại
                prvs = next
            s += 1

        print(videoPath + "---done--")


In [ ]:
labels = []
descriptor_list = []

Lấy đặc trưng và gắn nhãn cho từng clip

In [ ]:
path = "/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc"
extract_features(path, 0, descriptor_list, labels)
path = "/content/drive/My Drive/Machine Lol/Data/Của Trường/khongbaoluc"
extract_features(path, 1, descriptor_list, labels)

/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/136.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/137.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/138.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/139.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/140.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/141.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/142.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/143.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/144.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/145.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/146.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Của Trường/Cobaoluc/147.mp4---done--
/content/drive/M

##Chuẩn hóa đặc trưng

Chuẩn hóa giá trị của vector đặc trưng theo dạng minmax.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

trans = MinMaxScaler()

data = trans.fit_transform(descriptor_list)

##Lưu lại đặc trưng cho lần train tiếp theo

In [ ]:
import hickle as hkl

hkl.dump(descriptor_list, '/content/drive/My Drive/Machine Lol/backup/data3.hkl', mode='w')

hkl.dump(labels, '/content/drive/My Drive/Machine Lol/backup/labels3.hkl', mode='w')

#Training model

###Load Data Backup(Nếu có)

In [ ]:
import hickle as hkl
data = hkl.load('/content/data23.hkl')
labels = hkl.load('/content/labels23.hkl')

print(len(descriptor_list))
print(len(labels))

0
7191


Thống kê số nhãn thuộc 0 và 1

In [ ]:
print(len(labels) - sum(labels))
print(sum(labels))

3586
3605


Độ dài của vector đặc trưng của một example sau khỉ xử lý hog.

In [ ]:
print(len(data[1]))

54288


##Chia tỉ lệ Train|Test
Tỉ lệ giữa data train và data test là 0.8|0.2

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(data, labels,test_size =0.2, random_state=42)

##Model LogisticRegression

In [ ]:

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced', max_iter=1000)

lr.fit(trainX, trainY)


print(lr.score(trainX, trainY))
print(lr.score(testX, testY))

predY = lr.predict(testX)

score_LR = precision_recall_fscore_support(testY, predY, average='macro')

print(score_LR)

1.0
0.752605976372481
(0.754144144493474, 0.7533180667478308, 0.7525054592536766, None)


##Model GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(trainX, trainY)

print(gnb.score(trainX, trainY))
print(gnb.score(testX, testY))

predY = gnb.predict(testX)

score_GNB = precision_recall_fscore_support(testY, predY, average='macro')
print(score_GNB)

0.6623783031988874
0.610145934676859
(0.6106039201836483, 0.6089367499565193, 0.608098917183074, None)


##Model RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

rfc.fit(trainX, trainY)

print(rfc.score(trainX, trainY))
print(rfc.score(testX, testY))

predY = rfc.predict(testX)

score_RFC = precision_recall_fscore_support(testY, predY, average='macro')
print(score_RFC)

0.9895688456189151
0.6296038915913829
(0.6315593961064759, 0.6279822984907337, 0.626363953009721, None)


##Model SVM

In [ ]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(trainX, trainY))
print(lsvc.score(testX, testY))

predY = lsvc.predict(testX)

score_LSVC = precision_recall_fscore_support(testY, predY, average='macro')
print(score_LSVC)

0.9813977746870653
0.7866574009728978
(0.786569223455936, 0.7865847295495392, 0.7865765962925442, None)


In [ ]:
import pandas as pd
import numpy as np

scores_table = pd.DataFrame([np.array(score_LR), np.array(score_GNB), np.array(score_RFC), np.array(score_LSVC)])

scores_table.index = ["LogisticRegression", "GaussianNB", "RandomForestClassifier", "SVM"]
scores_table.columns = ["precision", "recall", "fscore", "support"]
print(scores_table)



                        precision    recall    fscore support
LogisticRegression       0.754144  0.753318  0.752505    None
GaussianNB               0.610604  0.608937  0.608099    None
RandomForestClassifier   0.631559  0.627982  0.626364    None
SVM                      0.786569  0.786585  0.786577    None


##Lưu Model

In [ ]:
import pickle

pickle.dump(lr, open('/content/drive/My Drive/ML/backup/logic.sav', 'wb'))
pickle.dump(gnb, open('/content/drive/My Drive/ML/backup/gauss.sav', 'wb'))
pickle.dump(rfc, open('/content/drive/My Drive/ML/backup/randforest.sav', 'wb'))
pickle.dump(lsvc, open('/content/drive/My Drive/ML/backup/svm.sav', 'wb'))

#Ứng dụng

###Download Model từ Google Drive

In [ ]:
#Train trên tập data1

#LogisticRegression
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1FwI7W1NRhXw7-vlgHgGifmB0pRKlmZVx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1FwI7W1NRhXw7-vlgHgGifmB0pRKlmZVx" -O logic.sav && rm -rf /tmp/cookies.txt

#GaussianNB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vdY64uXmi7HXwaazPb5oX86bj94l-MPg' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vdY64uXmi7HXwaazPb5oX86bj94l-MPg" -O gauss.sav && rm -rf /tmp/cookies.txt

#RandomForestClassifier
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1I84RrL1X9g33jlPeSGDTcPQTwUo7nFeU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1I84RrL1X9g33jlPeSGDTcPQTwUo7nFeU" -O randforest.sav && rm -rf /tmp/cookies.txt

#SVM
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3CPW1QfR4uVYOW8DQ9vmIRK20Lui7gC' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-3CPW1QfR4uVYOW8DQ9vmIRK20Lui7gC" -O svm.sav && rm -rf /tmp/cookies.txt


In [ ]:
#Train trên tập data2

#LogisticRegression
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-BS6UonLXs33RGfCUxYw_Ip84lRedT20' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-BS6UonLXs33RGfCUxYw_Ip84lRedT20" -O logic.sav && rm -rf /tmp/cookies.txt

#GaussianNB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-B64yg_JaNF5_Fywc-nLm_rSrPAFBOeu' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-B64yg_JaNF5_Fywc-nLm_rSrPAFBOeu" -O gauss.sav && rm -rf /tmp/cookies.txt

#RandomForestClassifier
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-ArwnLnUPjOMU9G4q-IuTTXT-Fn9sj_r' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-ArwnLnUPjOMU9G4q-IuTTXT-Fn9sj_r" -O randforest.sav && rm -rf /tmp/cookies.txt

#SVM
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P" -O svm.sav && rm -rf /tmp/cookies.txt


In [ ]:
#Train trên cả 2 tập data

#LogisticRegression
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-BS6UonLXs33RGfCUxYw_Ip84lRedT20' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-BS6UonLXs33RGfCUxYw_Ip84lRedT20" -O logic.sav && rm -rf /tmp/cookies.txt

#GaussianNB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-B64yg_JaNF5_Fywc-nLm_rSrPAFBOeu' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-B64yg_JaNF5_Fywc-nLm_rSrPAFBOeu" -O gauss.sav && rm -rf /tmp/cookies.txt

#RandomForestClassifier
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-ArwnLnUPjOMU9G4q-IuTTXT-Fn9sj_r' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-ArwnLnUPjOMU9G4q-IuTTXT-Fn9sj_r" -O randforest.sav && rm -rf /tmp/cookies.txt

#SVM
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P" -O svm.sav && rm -rf /tmp/cookies.txt


###Load Model

In [ ]:
import pickle

lr = pickle.load(open('/content/logic.sav', 'rb'))
gnb = pickle.load(open('/content/gauss.sav', 'rb'))
rfc = pickle.load(open('/content/randforest.sav', 'rb'))
lsvc = pickle.load(open('/content/svm.sav', 'rb'))



---



###Ứng dụng

In [ ]:
#@title Lựa chọn đầu vào
#@markdown Lựa chọn upload hay điền link video

Choose = "Link" #@param ["Upload", "Link", "---"]
#@markdown ---

try:
  lsvc
except NameError:
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-Gwy2tfeJxtUr6McbWpq--YFLzU4wP8P" -O svm.sav && rm -rf /tmp/cookies.txt
  import pickle
  lsvc = pickle.load(open('/content/svm.sav', 'rb'))
Key = True
list_fn = []
if Choose == "Upload":
  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('Người dùng đã upload file "{name}" độ lớn {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
    list_fn.append(fn)
elif Choose == "Link":
  print("Nhập link")
  link = input()

  import requests  
    
  r = requests.get(link, stream = True)  
  
  with open("/content/input.mp4", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
      if block:  
        file.write(block) 
  print("File đã được lưu với tên input.mp4")
  list_fn.append("input.mp4")

import numpy as np
import cv2
from skimage.feature import hog

num_vid = 0
while list_fn:
  i = list_fn.pop(0)
  cap = cv2.VideoCapture(i)

  ret, frame1 = cap.read()

  if ret:
    h, w, d = frame1.shape
    foname = 'output'+str(num_vid)+'.avi'
    out = cv2.VideoWriter(foname,cv2.VideoWriter_fourcc(*'DIVX'), 30, (w, h), True)
    flag = False
    frame1 = cv2.resize(frame1, (160, 120))
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    s = 1
    while(cap.isOpened()):
        ret, frame2 = cap.read()
        if not ret:
          break;
        if s % 6 == 0:
          frame = cv2.resize(frame2, (160, 120))
          #Trích xuất dòng quang từ 2 frame gần nhau
          next = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
          optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
          flow = optical_flow.calc(prvs, next, None)
          mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
          hsv[...,0] = ang*180/np.pi/2
          nom = np.zeros((160,120))
          hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
          bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
          gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
          
          v = np.median(gray)
          sigma = 0.33
          lower = int(max(0, (1.0 - sigma) * v))
          upper = int(min(255, (1.0 + sigma) * v))
          canny = cv2.Canny(gray, lower, upper)
          

          kernel = np.ones((5,5),np.uint8)
          closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

          contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

          for cnt in contour:
            cv2.drawContours(closing,[cnt],0,255,-1)

          opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)

          after_roi = next * opening

          # trích xuất feature bằng HOG
          image_hog = hog(after_roi, orientations=12, pixels_per_cell=(4, 4),
            cells_per_block=(2, 2), block_norm='L1', visualize=False)
          
          Y = lsvc.predict([image_hog])
          if Y[0] == 0:
            flag = True
          else:
            flag = False
        
          prvs = next.copy()
        if flag:
          cv2.rectangle(frame2,(0,0),(w,h),(0,0,255),10)
        out.write(frame2)

        s += 1
    
    out.release()
    print('Lưu kết quả của '+i+' ở file '+foname)

  num_vid += 1
if Choose == "---":
  print("Chọn một mục khác và thử lại!!!")
else:
  print("Hoan thanh!!!")

Nhập link
https://github.com/UITTrinhQuangTruong/CS114.K21/raw/master/test2.mp4
File đã được lưu với tên input.mp4
Lưu kết quả của input.mp4 ở file output0.avi
Hoan thanh!!!
